## Imports

In [1]:
from collections import Counter
from functools import reduce
from itertools import combinations
from math import floor, ceil

import re

## Day 1

In [2]:
with open('data/day01.txt', 'r+') as f:
    expense_report = [int(s) for s in f.read().split('\n') if s != '']

In [3]:
def expense_product(expense_report, n):
    ints = [c for c in combinations(expense_report, n) if sum(c) == 2020][0]
    return reduce((lambda x, y: x * y), ints)

* **Part 1**

In [4]:
expense_product(expense_report, 2)

1005459

* **Part 2**

In [5]:
expense_product(expense_report, 3)

92643264

## Day 2

In [6]:
with open('data/day02.txt', 'r+') as f:
    passwords_db = [s for s in f.read().split('\n') if s != '']

In [7]:
def count_valid_passwords(db, func):
    return sum([func(password) for password in db])

* **Part 1**

In [8]:
def is_valid_v1(password):
    rule, pwd = password.split(':')
    min_expected, max_expected = [int(x) for x in re.findall('\d+', rule)]
    letter = re.findall('[a-z]', rule)[0]
    actual = Counter(pwd.strip())[letter]
    return actual >= min_expected and actual < max_expected + 1

In [9]:
count_valid_passwords(passwords_db, is_valid_v1)

456

* **Part 2**

In [10]:
def is_valid_v2(password):
    rule, pwd = [s.strip() for s in password.split(':')]
    ix_1, ix_2 = [int(x) - 1 for x in re.findall('\d+', rule)]
    letter = re.findall('[a-z]', rule)[0]
    return (pwd[ix_1] == letter or pwd[ix_2] == letter) and not (pwd[ix_1] == letter and pwd[ix_2] == letter)

In [11]:
count_valid_passwords(passwords_db, is_valid_v2)

308

## Day 3

In [12]:
with open('data/day03.txt', 'r+') as f:
    tree_map = [list(s) for s in f.read().split('\n') if s != '']

In [13]:
def check_slope(tree_map, r, d):
    return sum(['#' == tree_map[i][j % len(tree_map[0])] 
                for i, j in zip(range(0,len(tree_map), d), range(0, len(tree_map) * r , r))])

* **Part 1**

In [14]:
check_slope(tree_map, 3, 1)

169

* **Part**

In [15]:
trees_encountered = [check_slope(tree_map, r, d) for r, d in [(1, 1), (3, 1), (5, 1), (7, 1), (1, 2)]]

In [16]:
reduce((lambda x, y: x * y), trees_encountered)

7560370818

## Day 4

In [17]:
with open("data/day04.txt") as f:
    lines = [line.rstrip('\n') for line in f]

In [18]:
def get_fields(doc):
    fields = {}
    for s in doc.split():
        k, v = s.split(':')
        fields[k] = v
    return(fields)

def get_documents(lines):
    empty_lines_index = [i for i, s in enumerate(lines) if s == ''] + [len(lines)]
    doc_strings = [' '.join(lines[ix[0]: ix[1]]) for ix in zip([0] + [i + 1 for i in empty_lines_index], empty_lines_index)]
    return [get_fields(doc) for doc in doc_strings]

* **Part 1**

In [19]:
def is_valid_document_v1(doc):
    return all([field in doc for field in ['byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid']])

In [20]:
sum([is_valid_document_v1(doc) for doc in get_documents(lines)])

226

* **Part 2**

In [21]:
def is_valid_byr(byr):
    return int(byr) >= 1920 and int(byr) <= 2002

def is_valid_iyr(iyr):
    return int(iyr) >= 2010 and int(iyr) <= 2020

def is_valid_eyr(eyr):
    return int(eyr) >= 2020 and int(eyr) <= 2030

def is_valid_hgt(hgt):
    results = re.findall('(\d+)(cm|in)', hgt)
    if results:
        n, unit = results[0]
        if unit == 'cm':
            return int(n) >= 150 and int(n) <= 193
        elif unit == 'in':
            return int(n) >= 59 and int(n) <= 76
    return False

def is_valid_hcl(hcl):
    return len(re.findall('#[aA-zZ0-9]{4}', hcl)) > 0

def is_valid_ecl(ecl):
    return ecl in ['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth']

def is_valid_pid(pid):
    return len(re.findall('\d', pid)) == len(pid)

In [22]:
def is_valid_document_v2(doc):
    if is_valid_document_v1(doc):
        return is_valid_byr(doc['byr']) and is_valid_iyr(doc['iyr'])\
            and is_valid_eyr(doc['eyr']) and is_valid_hgt(doc['hgt'])\
            and is_valid_hcl(doc['hcl']) and is_valid_ecl(doc['ecl'])\
            and is_valid_pid(doc['pid'])
    return False

In [23]:
sum([is_valid_document_v2(doc) for doc in get_documents(lines)])

162

## Day 5

In [24]:
with open("data/day05.txt") as f:
    boarding_passes = [line.rstrip('\n') for line in f]

In [25]:
def keep_lower_half(pair):
    return (pair[0], floor((pair[1] - pair[0]) / 2) + pair[0])

def keep_upper_half(pair):
    return (ceil((pair[1] - pair[0]) / 2) + pair[0], pair[1])

* **Part 1**

In [26]:
def get_row_and_col_number(boarding_pass):
    rows = (0, 127)
    cols = (0, 7)

    for char in boarding_pass:
        if char == 'F':
            rows = keep_lower_half(rows)
        elif char == 'B':
            rows = keep_upper_half(rows)
        elif char == 'L':
            cols = keep_lower_half(cols)
        elif char == 'R':
            cols = keep_upper_half(cols)
    
    if rows[0] == rows[1] and cols[0] == cols[1]:
        return rows[0], cols[0]

In [27]:
def get_seat_id(boarding_pass):
    row, col = get_row_and_col_number(boarding_pass)
    return row * 8 + col

In [28]:
max([get_seat_id(bp) for bp in boarding_passes])

965

* **Part 2**

In [29]:
ids = sorted([get_seat_id(bp) for bp in boarding_passes])

In [30]:
[(i, j, j - i) for i, j in zip(ids[:-1], ids[1:]) if j - i > 1]

[(523, 525, 2)]

## Day 6

In [31]:
with open('data/day06.txt', 'r+') as f:
    groups = f.read().split('\n\n')

* **Part 1**

In [32]:
sum([len(set(g.replace('\n', ''))) for g in groups])

6630

* **Part 2**

In [33]:
def count_group_answers(group):
    n_people = len([a for a in group.split('\n') if a != ''])
    return sum([1 for c in Counter(group.replace('\n', '')).values() if c == n_people])

In [34]:
sum([count_group_answers(g) for g in groups])

3437